# Testing merges before turning them into functions

In [1]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import fuzzymatcher
pd.options.display.max_rows = 250

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_vehiclesdata()
organizations= data_prep.load_organizations_data()

/home/jovyan/data-analyses/5311/data_prep.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Merge NTD with GTFS  --> m2_df Function


In [3]:
#reading in crosswalk 1
crosswalk1 = pd.read_parquet('test_crosswalk_both.parquet', engine='pyarrow')

In [4]:
len(crosswalk1)

44

In [5]:
#rename to agency so we can match it
crosswalk1 = crosswalk1.rename(columns = {'agency_x':'agency'})

In [6]:
#first crosswalk
m1_df = vehicles.merge(crosswalk1, on=['agency','ntd_id'],  how='outer', indicator=True)

In [7]:
m1_df['agency'].nunique()

52

In [8]:
m1_df = m1_df.drop(columns = ['_merge'])

In [9]:
#reading crosswalk 2 (the manual one that captures some of the missing values)
crosswalk2 = pd.read_csv('vehicles-left-only.csv')

In [10]:
crosswalk2 

,agency,ntd_id,itp_id
0,San Benito County LTA,9R02-91009,274.0
1,County of Sacramento Municipal Services Agency...,9R02-90216,NaN
2,"City of Dixon, dba: Readi-Ride",9R02-91041,94.0
3,"Mariposa County Transit, dba: Mari-Go",9R02-91082,NaN
4,Calaveras Transit Agency,9R02-99442,NaN
5,Palo Verde Valley Transit Agency,9R02-99454,238.0
6,"City of Ridgecrest, dba: Ridgerunner",9R02-91006,263.0
7,Modoc Transportation Agency,9R02-91008,NaN
8,"Wasco, City of",9R02-99426,363.0


In [11]:
#second crosswalk
m2_df = m1_df.merge(crosswalk2, on=['agency', 'ntd_id','itp_id'],  how='outer')

In [12]:
m2_df['agency'].nunique()

52

In [13]:
#merging the 2 datasets together
vehicles_gtfs = m2_df.merge(organizations, on=['agency', 'ntd_id', 'itp_id'],  how='left', indicator=True)

In [14]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

In [15]:
vehicles_gtfs['agency'].nunique()

52

In [16]:
vehicles_gtfs.head(2)

,agency,city,state,legacy_ntd_id,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,vehicle_type,years_old:,_13_15,_16_20,_21_25,_26_30,_31_60,_60+,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,_0_9,_10_12,vehicle_groups,itp_id,gtfs_schedule_status,#_services_w__complete_rt_status,#_fixed_route_services_w__static_gtfs,complete_static_gtfs_coverage__1=yes_,complete_rt_coverage,>=1_gtfs_feed_for_any_service__1=yes_,>=_1_complete_rt_set__1=yes_
0,Fresno County Rural Transit Agency,Fresno,CA,9R02-025,9R02-91007,Independent Public Agency or Authority of Tran...,Rural Reporter,0.0,77.0,Van,NaN,0.0,0.0,0.0,0.0,0.0,0.0,18.0,2.555556,0.0,18.0,0.0,Vans,117.0,"needed,needed,ok,ok,needed,ok,ok,needed,needed...",0.0,4.0,0.0,0.0,1.0,0.0
1,Fresno County Rural Transit Agency,Fresno,CA,9R02-025,9R02-91007,Independent Public Agency or Authority of Tran...,Rural Reporter,0.0,77.0,Trucks and other Rubber Tire Vehicles (Service),NaN,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,NaN,3.0,0.0,Vans,117.0,"needed,needed,ok,ok,needed,ok,ok,needed,needed...",0.0,4.0,0.0,0.0,1.0,0.0


In [17]:
vehicles['agency'].sort_values().unique()

array(['Alpine County Local Transportation Commission',
       'Amador Regional Transit System', 'Calaveras Transit Agency',
       'City of Arcata', 'City of Arvin', 'City of Auburn',
       'City of California City',
       'City of Chowchilla, dba: Chowchilla Area Transit ',
       'City of Corcoran, dba: Corcoran Area Transit', 'City of Dinuba',
       'City of Dixon, dba: Readi-Ride', 'City of Escalon, dba: eTrans',
       'City of Eureka, dba: Eureka Transit Service', 'City of Guadalupe',
       'City of McFarland', 'City of Needles, dba: Needles Area Transit',
       'City of Ojai', 'City of Ridgecrest, dba: Ridgerunner',
       'City of Rio Vista, dba: Rio Vista Delta Breeze',
       'City of Shafter, dba: Shafter Dial-A-Ride', 'City of Solvang',
       'City of Taft, dba: Taft Area Transit', 'City of Woodlake',
       'Colusa County Transit Agency',
       'County of Sacramento Municipal Services Agency Department of Transportation',
       'County of Siskiyou, dba: Siskiyou C

# Merging Black Cat with vehicles_gtfs

In [18]:
right_on = ["agency"]
left_on = ["organization_name"]

In [19]:
m4_df = fuzzymatcher.fuzzy_left_join(df_5311, vehicles_gtfs, left_on, right_on)

In [20]:
#stuff to filter out before cutting off at scores...
subset3 = ['Plumas County Transportation Commission', 'Imperial County Transportation Commission', 'Glenn County Transportation Commission','Tehama County Transit Agency','Tuolumne County Transit Agency (TCTA)']

In [21]:
#filter out a bunch of stuff all at once
m4_df = m4_df[~m4_df.organization_name.isin(subset3)]

In [22]:
#filter out for any scores below 0.075
m5_df = m4_df.loc[m4_df['best_match_score'] > 0.075 ]

In [23]:
#total agencies that matched over
m5_df['organization_name'].nunique()

43

In [24]:
m5_df.shape

(401, 49)

# Walking back in the other 45 agencies that were cut

In [25]:
#reading crosswalk 3, what I manually inputted
crosswalk3 = pd.read_excel('BlackCat_NTD_CalITP.xlsx')

In [26]:
#crosswalk on the left
unmatched = crosswalk3.merge(df_5311, on=['organization_name'],  how='left')

In [27]:
unmatched['agency'].nunique()

34

In [28]:
unmatched.head(1)

,organization_name,agency,ntd_id,grant_fiscal_year,funding_program,grant_number,project_year,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status
0,Madera County,"Madera County, dba: Madera County Connection T...",9R02-91005,2011.0,Section 5311,CA-18-X047 | 0012000083,2016.0,BCG0000283,Buy <30-Ft Bus For Expansion,111304,64BC17-00408,110663.0,110663.0,101352.02,9310.98,0.0,Open


In [29]:
#reading in vehicle without filtering for reporter type...test
vehicles_info =  pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020-Vehicles_1.xlsm', sheet_name = 'Age Distribution')

In [30]:
vehicles_info = vehicles_info.rename(columns = {'Agency':'agency', 'NTD ID':'ntd_id'})

In [31]:
vehicles_info.head(1)

,agency,City,State,Legacy NTD ID,ntd_id,Organization Type,Reporter Type,Primary UZA Population,Agency VOMS,Vehicle Type,Years Old:,0,1,2,3,4,5,6,7,8,9,10,11,12,13-15,16-20,21-25,26-30,31-60,60+,Total Vehicles,Average Age Of Fleet (In Years),Average Lifetime Miles Per Vehicle
0,MTA New York City Transit,Brooklyn,NY,2008,20008,"Subsidiary Unit of a Transit Agency, Reporting...",Full Reporter,18351295,10427,Van,NaN,0,32,0,2,10,0,4,0,0,0,0,0,0,0,0,0,0,0,0,48,2.125,70685.0417


In [32]:
#joining vehicles-gtfs data frame
unmatched2 = unmatched.merge(vehicles_info, on=['agency','ntd_id'],  how='left', indicator=True)

In [33]:
unmatched2._merge.value_counts()

both          1523
left_only       54
right_only       0
Name: _merge, dtype: int64

In [37]:
unmatched2.sample(5)

,organization_name,agency,ntd_id,grant_fiscal_year,funding_program,grant_number,project_year,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,City,State,Legacy NTD ID,Organization Type,Reporter Type,Primary UZA Population,Agency VOMS,Vehicle Type,Years Old:,0,1,2,3,4,5,6,7,8,9,10,11,12,13-15,16-20,21-25,26-30,31-60,60+,Total Vehicles,Average Age Of Fleet (In Years),Average Lifetime Miles Per Vehicle,_merge
1278,Sunline Transit Agency,SunLine Transit Agency,90079,2019.0,5311(f) Round 2,CA-2020-015 | 0020000057-F,2019.0,BCG0001612,Operating Assistance Sliding Scale,300902,64CO19-01012,185187.0,185187.0,185187.0,0.0,0.0,Open,Thousand Palms,CA,9079,Independent Public Agency or Authority of Tran...,Full Reporter,345580.0,98.0,Sports Utility Vehicle,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,41939.0,both
210,San Diego Metropolitan Transit System,San Diego Metropolitan Transit System,90026,2019.0,Section 5311,CA-2020-015 | 0020000057,2019.0,BCG0001513,Operating Assistance Sliding Scale,300902,64BO19-00977,150000.0,150000.0,150000.0,0.0,0.0,Open,San Diego,CA,9026,Independent Public Agency or Authority of Tran...,Full Reporter,2956746.0,906.0,Automobiles (Service),NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,both
150,San Diego Metropolitan Transit System,San Diego Metropolitan Transit System,90026,2014.0,Section 5311,CA-18-X069 | 0015000075,2017.0,BCG0001128,Operating Assistance Sliding Scale,300902,64BO17-00579,131388.0,275000.0,131388.0,0.0,0.0,Open,San Diego,CA,9026,Independent Public Agency or Authority of Tran...,Full Reporter,2956746.0,906.0,Over-the-road Bus,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,24.0,13.0,440302.0,both
1329,Transit Joint Powers Authority for Merced County,Transit Joint Powers Authority for Merced Coun...,90173,2018.0,Section 5311,CA-2018-114 | 0019000012,2018.0,BCG0001183,Operating Assistance Sliding Scale,300902,64BO19-00878,507341.0,507341.0,507341.0,0.0,0.0,Open,Merced,CA,9173,Independent Public Agency or Authority of Tran...,Full Reporter,136969.0,53.0,Trucks and other Rubber Tire Vehicles (Service),NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,NaN,NaN,both
265,City of Fairfield,"City of Fairfield, California, dba: Fairfield ...",90092,2016.0,Section 5311,CA-2017-025 | 0017000158,2016.0,BCG0000231,Operating Assistance,300902,64BO17-00374,100000.0,100000.0,100000.0,0.0,0.0,Open,Fairfield,CA,9092,"City, County or Local Government Unit or Depar...",Full Reporter,133683.0,39.0,Trucks and other Rubber Tire Vehicles (Service),NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,NaN,NaN,both


In [35]:
#unmatched2.to_csv("./unmatched2.csv", index = False) 